# =========================Libraries ============================

In [1]:
import gym 
import numpy as np 
from lib.envs.gridworld import GridworldEnv

# ======================== Parameters ==========================

In [2]:
gamma = 0.9
theta = 10**-5

# ======================== Functions ==========================

In [3]:
def one_step_lookahead(state,V):
    """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
    """
    
    Q_s = np.zeros(env.nA)
    for a in range(env.nA):
        v_s_a = 0
        for prob,next_s,R,done in env.P[state][a]:
            v_s_a += prob*gamma*(R+V[next_s])
            
        Q_s[a] = v_s_a
        
    return Q_s

In [4]:
def policy_eval(policy,env,gamma,theta):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    
    V = np.zeros(env.nS)
    while True:
        delta = 0
        
        for s in range(env.nS):
            v_s = 0
            
            for a,p_a in enumerate(policy[s].tolist()):
                
                v_s_a = 0
                for prob,next_s,R,done in env.P[s][a]:
                    v_s_a += prob*gamma*(R+V[next_s])
                    
                v_s += p_a*v_s_a
            delta = max(delta,np.abs(V[s]-v_s))
            V[s]  = v_s
        if delta<theta:
            break
    return V

In [5]:
def policy_improvment(env,gamma,theta):
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        policy_stable = True
        
        V = policy_eval(policy,env,gamma,theta)
        for s in range(env.nS):
            Q_s = one_step_lookahead(s,V)
            new_action = np.argmax(Q_s)
            
            old_action = np.argmax(policy[s])
            
            if old_action!=new_action:
                policy_stable = False
                
            policy[s] = np.eye(env.nA)[new_action]
            
        if policy_stable:
            return policy, V 

# =========================== Main =============================

In [6]:
env = GridworldEnv()

In [7]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env,gamma,theta)

In [8]:
policy, v = policy_improvment(env,gamma,theta)

In [9]:
policy

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [10]:
v

array([ 0.   , -0.9  , -1.71 , -2.439, -0.9  , -1.71 , -2.439, -1.71 ,
       -1.71 , -2.439, -1.71 , -0.9  , -2.439, -1.71 , -0.9  ,  0.   ])